In [1]:
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import pandas as pd
import seaborn as sn

plt.style.use("ggplot")

# Update annual copper from Aquaculture

Each year, we receive national figures for the amount of copper used by the aquaculture industry to treat nets etc. However, in recent years, we have reused figures from previous years as the correct annual figure was not available in time for the analysis.

This notebook updates the database to use the "official" figures for 2016 to 2020.

In [2]:
eng = nivapy.da.connect()

Username:  ···
Password:  ···············


Connection successful.


In [3]:
xl_path = r"../../../Data/annual_copper_usage_aquaculture.xlsx"
ann_cu_df = pd.read_excel(xl_path, sheet_name="Sheet1")
ann_cu_df = ann_cu_df.query("year >= 2016")
ann_cu_df.head()

,year,tot_cu_tonnes
26,2016,1335
27,2017,1432
28,2018,1626
29,2019,1698
30,2020,1539


In [4]:
df_list = []
for idx, row in ann_cu_df.iterrows():
    year = row["year"]
    cu_loss = 0.85 * row["tot_cu_tonnes"]

    sql = (
        "SELECT * FROM resa2.rid_kilder_aqkult_values "
        "WHERE ar = %s"
        "AND inp_par_id = 40" % year
    )
    p_df = pd.read_sql(sql, eng)

    p_df["value"] = cu_loss * p_df["value"] / p_df["value"].sum()
    p_df["inp_par_id"] = 41
    del p_df["value_id"]

    df_list.append(p_df)

    print(int(p_df["value"].sum()), int(cu_loss))

df = pd.concat(df_list)

df.head()

1134 1134
1217 1217
1382 1382
1443 1443
1308 1308


,anlegg_nr,inp_par_id,ar,maned,art,value
0,36099,41,2016,6,None,0.237773
1,36100,41,2016,6,None,0.464830
2,36117,41,2016,6,None,0.591945
3,36118,41,2016,6,None,0.171812
4,36177,41,2016,6,None,0.425229


In [5]:
# df.to_sql(
#     "rid_kilder_aqkult_values",
#     con=eng,
#     schema="resa2",
#     if_exists="append",
#     index=False,
# )